# Distance based time series classification in aeon

Distance based classifiers use a distance function to measure the similarity
between time series. Historically, distance functions have been mostly used with
a nearest neighbor (NN) classifiers. 1-NN with dynamic time warping is often used as
a benchmark, although it is no longer competitive. Aeon provides a range of time series
specific distance measures in the distances module (see LINK distances notebook).
These can all be used



## Load data and list distance based classifiers

In [2]:
from sktime.datasets import load_basic_motions, load_italy_power_demand

X_train, y_train = load_italy_power_demand(split="train", return_X_y=True)
X_test, y_test = load_italy_power_demand(split="test", return_X_y=True)
X_test = X_test[:50]
y_test = y_test[:50]

print(X_train.shape, y_train.shape, X_test.shape, y_test.shape)

X_train_mv, y_train_mv = load_basic_motions(split="train", return_X_y=True)
X_test_mv, y_test_mv = load_basic_motions(split="test", return_X_y=True)

X_train_mv = X_train_mv[:20]
y_train_mv = y_train_mv[:20]
X_test_mv = X_test_mv[:20]
y_test_mv = y_test_mv[:20]

print(X_train_mv.shape, y_train_mv.shape, X_test_mv.shape, y_test_mv.shape)

(67, 1) (67,) (50, 1) (50,)
(20, 6) (20,) (20, 6) (20,)


In [3]:
from sktime.registry import all_estimators

# search for all classifiers that can handle multivariate time series. This will
# give some UserWarnings if soft dependencies are not installed.
all_estimators(
    filter_tags={"classifier_type": "distance"}, estimator_types="classifier"
)

[('ElasticEnsemble',
  sktime.classification.distance_based._elastic_ensemble.ElasticEnsemble),
 ('MatrixProfileClassifier',
  sktime.classification.feature_based._matrix_profile_classifier.MatrixProfileClassifier),
 ('ShapeDTW', sktime.classification.distance_based._shape_dtw.ShapeDTW)]

## 1-NN with elastic distances and the elastic ensemble




In [ ]:
from sktime.classification.distance_based import (
    ElasticEnsemble,
    KNeighborsTimeSeriesClassifier,
    ShapeDTW,
)

knn = KNeighborsTimeSeriesClassifier(distance="dtw")
shape = ShapeDTW()
ee = ElasticEnsemble()

## References

